In [ ]:
import xarray as xr
import numpy as np
import sys
sys.path.append('/home/563/rl5183/PhD-research/Functions')
import functions as func
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import dask
dask.config.set({'array.slicing.split_large_chunks': False})

In [ ]:
# Generate dictionary of all model names and file paths 
import os
import pandas as pd
    
var = 'txx'
ensemble = 'r1i1p1f1'

institution_dir = '/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccdi/data/v1-0/etccdi/base_independent/mon/historical'

all_models = os.listdir(institution_dir)

var_files = []
model_names = []

for model in all_models:
    all_files = os.listdir(f'{institution_dir}/{model}')
    for file in all_files:
        if var in file and ensemble in file:
            var_files.append(f'{model}/{file}')
            model_names.append(model)

paths = [f'/g/data/ia39/aus-ref-clim-data-nci/cmip6-etccdi/data/v1-0/etccdi/base_independent/mon/historical/{f}'
             for f in var_files]

dictionary = {model_names[i]: paths[i] for i in range(len(model_names))}

In [ ]:
# Read in each file and save to my directy for regridding 
names = []

for name, path in dictionary.items():
    try:
        d = xr.open_mfdataset(path, combine='by_coords', chunks={'time':-1, 'lat':110, 'lon':110}).txxETCCDI
        if len(d['time'])!=1980:
            d = d.sel(time=slice('1850-01', '2014-12'))
        del d['time']
        if hasattr(d, 'height'):
            del d['height']
        time_month = pd.date_range(start='1850-01',end = '2015-01', freq ='M')
        d.coords['time'] = time_month
        d.to_netcdf(f'/g/data/w40/rl5183/CMIP6_ETCCDI/{name}_txx')
        print(f'Model {name} saved')
        names.append(name)
    except OSError:
        # No files read, move on to the next
        continue 